# Shut the Box

## Step 0: Utils for Jupyter Notebooks

### Add button to Show/Hide code (to focus on outputs)

In [23]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Show Code"></form>''')

### Add magic command to store python variables in csv file

In [45]:
# credit to sirex, https://github.com/ipython/ipython/issues/6701
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

## Step 1: Generate models

In [66]:
# Edit this cell to define parameters for your model...

boards = 12 # Number of boards to use
sides = 6 # Number of sides per die
die = 2 # Number of die to roll each round

max_score = boards * (boards+1) // 2

In [67]:
%%writetemplate ShutTheBox/params/stb_params_high.csv
b, {boards}
d, {sides}
ndie, {die}
strategy, 1

In [68]:
%%writetemplate ShutTheBox/params/stb_params_low.csv
b, {boards}
d, {sides}
ndie, {die}
strategy, 2

In [69]:
%%writetemplate ShutTheBox/params/stb_nondet_params.csv
b, {boards}
d, {sides}
ndie, {die}
strategy, 0

In [70]:
!mkdir test

A subdirectory or file test already exists.


In [71]:
model_spec = f"stb{boards}_{die}d{sides}"

!mkdir "ShutTheBox/models/$model_spec"
output_model_high = f"ShutTheBox/models/{model_spec}/{model_spec}_high.prism"

!python utils/pyprism.py ShutTheBox/stb.pyprism $output_model_high ShutTheBox/params/stb_params_high.csv

A subdirectory or file ShutTheBox/models/stb12_2d6 already exists.


Model generated successfully at ShutTheBox\models\stb12_2d6\stb12_2d6_high.prism


In [72]:
output_model_low = f"ShutTheBox/models/{model_spec}/{model_spec}_low.prism"

!python utils/pyprism.py ShutTheBox/stb.pyprism $output_model_low ShutTheBox/params/stb_params_low.csv

Model generated successfully at ShutTheBox\models\stb12_2d6\stb12_2d6_low.prism


In [73]:
output_model_nondet = f"ShutTheBox/models/{model_spec}/{model_spec}_nondet.prism"

!python utils/pyprism.py ShutTheBox/stb.pyprism $output_model_nondet ShutTheBox/params/stb_nondet_params.csv

Model generated successfully at ShutTheBox\models\stb12_2d6\stb12_2d6_nondet.prism


In [74]:
output_props = f"ShutTheBox/props/{model_spec}.props"

!python utils/pyprism.py ShutTheBox/stbprops.pyprism $output_props ShutTheBox/params/stb_params_high.csv

Model generated successfully at ShutTheBox\props\stb12_2d6.props


## Step 2: Check properties, run experiments, export data

### Does the game eventually always terminate?

We expect to see a value of approximately 1 here (it won't be exact because CSGs only support methods based on numerical convergence, so it may be slightly under or even over 1!

In [75]:
!prism $output_model_high $output_props -prop 1 | grep -E "(Model checking)|(Result)|(sec)|(Error)"

Model checking: <<p1>>Pmax=? [ F game_over ]
Reachable states exploration and model construction done in 5.379 secs.
Time for model construction: 5.536 seconds.
Precomputation took 0.895 seconds.
Time for model checking: 3.597 seconds.
Result: 1.0000000000000002 (value in the initial state)


### Probability of obtaining each score 

In [76]:
data_location = f"ShutTheBox/data/{model_spec}"

!mkdir "$data_location"

prob_score_high_file = f"{data_location}/{model_spec}_prob_score_high.csv"

!prism $output_model_high $output_props -prop 2 -const k=1:{max_score} -exportresults {prob_score_high_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(Error)"

A subdirectory or file ShutTheBox/data/stb12_2d6 already exists.


Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=2
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=5
Result: 2.5720164609053495E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=6
Result: 3.858024691358024E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=7
Result: 9.430727023319616E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=8
Result: 0.001178840877914952

In [77]:
prob_score_low_file = f"{data_location}/{model_spec}_prob_score_low.csv"

!prism $output_model_low $output_props -prop 2 -const k=1:{max_score} -exportresults {prob_score_low_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(sec)|(Error)"

Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=1
Reachable states exploration and model construction done in 0.836 secs.
Time for model construction: 0.875 seconds.
Precomputation took 0.031 seconds.
Time for model checking: 0.073 seconds.
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=2
Precomputation took 0.128 seconds.
Time for model checking: 0.158 seconds.
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=3
Precomputation took 0.135 seconds.
Time for model checking: 0.147 seconds.
Result: 0.0015432098765432098 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=4
Precomputation took 0.108 seconds.
Time for model checking: 0.119 seconds.
Result: 0.004629629629629629 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Propert

In [78]:
prob_score_nondet_min_file = f"{data_location}/{model_spec}_prob_score_nondet_min.csv"

!prism $output_model_nondet $output_props -prop 2 -const k=1:{max_score} -exportresults {prob_score_nondet_min_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(Error)"

Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=2
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=5
Result: 1.2860082304526747E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=6
Result: 3.858024691358024E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=7
Result: 4.715363511659808E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=8
Result: 0.001071673525377229

In [79]:
prob_score_nondet_max_file = f"{data_location}/{model_spec}_prob_score_nondet_max.csv"

!prism $output_model_nondet $output_props -prop 3 -const k=1:{max_score} -exportresults {prob_score_nondet_max_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(Error)"

Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=2
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=3
Result: 0.0015432098765432098 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=4
Result: 0.004629629629629629 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=5
Result: 0.009516460905349794 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=6
Result: 0.012217078189300411 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=7
Result: 0.020233196159122082 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants

### Expected number of die rolls

In [80]:
print("High board strategy:")
!prism $output_model_high $output_props -prop 9 | grep -E "(Model checking)|(Result)|(Error)"
print("Low board strategy:")
!prism $output_model_low $output_props -prop 9 | grep -E "(Model checking)|(Result)|(Error)"

High board strategy:
Model checking: <<p1>>R{"no_rolls"}max=? [ F game_over ]
Result: 7.006172038789394 (value in the initial state)
Low board strategy:
Model checking: <<p1>>R{"no_rolls"}max=? [ F game_over ]
Result: 4.219568128753051 (value in the initial state)


### Probability of obtaining a score given a particular board is covered

In [81]:
for covered_board in range(1, boards+1):
    !prism $output_model_high $output_props -prop {10+covered_board} -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_score_with_covered_{covered_board}.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=6
Result: 2.5720164609053495E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=7
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=8
Result: 9.645061728395061E-4 (value in 

Model checking: <<p1>>Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=2
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=5
Result: 2.5720164609053495E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=6
Result: 3.858024691358024E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=7
Result: 1.7146776406035664E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b2=1&score=k ]
Property co

Model checking: <<p1>>Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=5
Result: 2.5720164609053495E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=6
Result: 2.5720164609053495E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=7
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=8
Resu

Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=6
Result: 1.2860082304526747E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=7
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=8
Result: 9.645061728395

Model checking: <<p1>>Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=6
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=7
Result: 1.7146776406035664E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=8
Result: 0.0 (value in the initial state

...

Model checking: <<p1>>Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=6
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=7
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=8
Result: 0.0 (value in the initial state)
Model checking: 

Model checking: <<p1>>Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=6
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=7
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=8
Result: 0.0 (value in the initial state)
Model checking: 

Model checking: <<p1>>Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=6
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=7
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=8
Result: 0.0 (value in the initial state)
Model ch

Model checking: <<p1>>Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=6
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=7
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=8
Result: 0.0 (value in the initial state)
Model ch

Model checking: <<p1>>Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=6
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=7
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=8
Result: 0.0 (value in the initial state)
Model ch

### Cumulative probability of a score

In [82]:
!prism $output_model_high $output_props -prop 6 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_high.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=2
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=3
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=4
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.0010288065843621398 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.0014146090534979422 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.0023576817558299042 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property

In [83]:
!prism $output_model_low $output_props -prop 6 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_low.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=2
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=3
Result: 0.0023148148148148147 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=4
Result: 0.006944444444444444 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.01633230452674897 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.028549382716049378 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.04792524005486968 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property con

In [84]:
!prism $output_model_nondet $output_props -prop 5 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_min.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=2
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=3
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=4
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.0010288065843621398 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.0014146090534979422 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.002357681755829904 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property 

In [85]:
cum_prob_max_file = "ShutTheBox/data/cum_prob_max.csv"

!prism $output_model_nondet $output_props -prop 6 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_max.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=2
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=3
Result: 0.0023148148148148147 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=4
Result: 0.006944444444444444 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.01633230452674897 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.028549382716049378 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.0482681755829904 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property cons